In [2]:
# 스마트스토어 후기 크롤링(엑셀로구글드라이브에저장버전)

import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import os
import subprocess
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException, NoSuchWindowException
from datetime import datetime
import sys

# Google Drive API 관련 라이브러리 추가
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# 엑셀 처리를 위한 추가 라이브러리
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 하드 코딩된 구글 드라이브 폴더 ID
GOOGLE_DRIVE_FOLDER_ID = "18SbGtVGs19fYoe-XNajnnl3IlLrP5R2u"

# 엑셀 파일에서 입력 읽기 (첫 번째 행 건너뛰기)
def read_excel_input(file_path):
    try:
        # 엑셀 파일에서 데이터를 읽고, 첫 번째 행을 열 이름으로 처리
        df = pd.read_excel(file_path)
        
        # 공백 제거
        df.columns = df.columns.str.strip()

        # 실제로 읽어온 열 이름을 로깅하여 확인
        logging.info(f"엑셀 파일에서 읽어온 열 이름: {df.columns.tolist()}")

        required_columns = ['상품 페이지 URL', '파일 이름', '정렬 옵션', '최대 리뷰 수']
        
        # 필요한 열이 엑셀 파일에 모두 존재하는지 확인
        if not all(col in df.columns for col in required_columns):
            raise ValueError("엑셀 파일에 필요한 열이 모두 존재하지 않습니다.")
        
        inputs = []
        for _, row in df.iterrows():
            url = row['상품 페이지 URL']
            file_name = row['파일 이름']
            sort_option = row['정렬 옵션']
            max_reviews = row['최대 리뷰 수']
            
            if not isinstance(url, str) or 'naver.com' not in url:
                raise ValueError(f"잘못된 URL 형식: {url}")
            
            if not isinstance(file_name, str):
                raise ValueError(f"잘못된 파일 이름: {file_name}")
            
            if sort_option not in [1, 2, 3, 4]:
                raise ValueError(f"잘못된 정렬 옵션: {sort_option}")
            
            if not isinstance(max_reviews, (int, float)) or max_reviews <= 0:
                raise ValueError(f"잘못된 최대 리뷰 수: {max_reviews}")
            
            inputs.append((url, file_name, int(sort_option), int(max_reviews)))
        
        return inputs
    
    except Exception as e:
        logging.error(f"엑셀 파일 읽기 오류: {e}")
        return None


# 엑셀 파일 상태 업데이트
def update_excel_status(file_path, row_index, status):
    try:
        wb = load_workbook(file_path)
        sheet = wb.active
        status_cell = sheet.cell(row=row_index + 2, column=len(sheet[1]) + 1)
        status_cell.value = status
        if status == "완료":
            status_cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
        elif status == "실패":
            status_cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
        wb.save(file_path)
    except Exception as e:
        logging.error(f"엑셀 파일 업데이트 오류: {e}")

# 웹드라이버 초기화 함수
def initialize_webdriver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# 랜덤 대기 시간 함수
def random_wait(min_time=2, max_time=4):
    time.sleep(random.uniform(min_time, max_time))

# 새로고침 버튼 클릭 함수 추가
def check_and_click_refresh_button(driver):
    try:
        refresh_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "button highlight") and contains(text(), "새로고침")]'))
        )
        if refresh_button:
            logging.info("새로고침 버튼이 감지되었습니다. 버튼을 클릭합니다.")
            refresh_button.click()
            random_wait()
            logging.info("페이지가 새로고침되었습니다.")
    except TimeoutException:
        logging.info("새로고침 버튼이 감지되지 않았습니다. 계속 진행합니다.")
    except NoSuchElementException:
        logging.info("새로고침 버튼을 찾을 수 없습니다. 계속 진행합니다.")

# 리뷰 정렬 옵션 설정 함수
def set_review_sort(driver, sort_option):
    try:
        sort_options_xpath = {
            1: '//a[contains(text(), "랭킹순")]',
            2: '//a[contains(text(), "최신순")]',
            3: '//a[contains(text(), "평점 높은순")]',
            4: '//a[contains(text(), "평점 낮은순")]'
        }
        sort_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, sort_options_xpath[sort_option]))
        )
        sort_button.click()
        logging.info(f"선택한 정렬 옵션: {['랭킹순', '최신순', '평점 높은순', '평점 낮은순'][sort_option - 1]}")
        random_wait()
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"정렬 버튼을 클릭할 수 없습니다. 오류: {e}")

# 리뷰 데이터를 중간 저장하는 함수
def save_to_csv_partial(reviews_data, file_name, sort_option, part_number):
    sort_options = {1: "랭킹순", 2: "최신순", 3: "평점높은순", 4: "평점낮은순"}
    sort_option_name = sort_options[sort_option]
    
    current_date = datetime.now().strftime("%Y-%m-%d")
    final_file_name = f"{file_name}_스마트스토어후기_{sort_option_name}_{current_date}_part{part_number}"
    
    df = pd.DataFrame(reviews_data)
    df.index += 1
    full_path = os.path.abspath(f"{final_file_name}.csv")
    df.to_csv(full_path, index_label="순번", encoding="utf-8-sig")
    logging.info(f"{full_path} 파일에 중간 저장되었습니다.")
    return full_path

# Google Drive 업로드 함수
def upload_to_google_drive(file_path, folder_id):
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': os.path.basename(file_path), 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    try:
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        logging.info(f"파일이 구글 드라이브에 업로드되었습니다. File ID: {file.get('id')}")
    except Exception as e:
        logging.error(f"구글 드라이브 업로드 실패: {e}")

# 리뷰 데이터를 크롤링하는 함수
def extract_review_info(driver, reviews_data, max_reviews, file_name, sort_option):
    page_num = 1
    part_number = 1
    while len(reviews_data) < max_reviews:
        try:
            logging.info(f"현재 페이지: {page_num}")
            
            reviews = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.BnwL_cs1av'))
            )
            
            logging.info(f"현재 페이지에서 발견된 리뷰 수: {len(reviews)}")
            
            for review in reviews:
                if len(reviews_data) >= max_reviews:
                    break
                try:
                    rating = review.find_element(By.CSS_SELECTOR, 'em._15NU42F3kT').text
                    user_id = review.find_element(By.CSS_SELECTOR, 'strong._2L3vDiadT9').text
                    raw_date = review.find_element(By.CSS_SELECTOR, 'div.iWGqB6S4Lq span._2L3vDiadT9').text

                    raw_date = raw_date.split('.')[0:3]
                    date_str = '.'.join(raw_date)
                    date = datetime.strptime(date_str, "%y.%m.%d").strftime("%Y-%m-%d")
                    
                    option = review.find_element(By.CSS_SELECTOR, 'div._2FXNMst_ak').text
                    review_text = review.find_element(By.CSS_SELECTOR, 'div._1kMfD5ErZ6 span._2L3vDiadT9').text
                    
                    reviews_data.append({
                        "평점": rating,
                        "구매자아이디": user_id,
                        "날짜": date,
                        "선택옵션명": option,
                        "리뷰내용": review_text
                    })
                    logging.info(f"리뷰 {len(reviews_data)}: {user_id} - {rating}")
                except NoSuchElementException as e:
                    logging.warning(f"리뷰 요소를 찾을 수 없습니다. 오류: {e}")
                    continue

            if len(reviews_data) % 100 == 0:
                save_to_csv_partial(reviews_data, file_name, sort_option, part_number)
                part_number += 1

            try:
                next_page_buttons = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.UWN4IvaQza._nlog_click[aria-current="false"]'))
                )
                
                next_page_button = None
                for button in next_page_buttons:
                    button_number = int(button.text)
                    if button_number > page_num:
                        next_page_button = button
                        break
                
                if next_page_button:
                    driver.execute_script("arguments[0].click();", next_page_button)
                    page_num = button_number
                    random_wait()
                else:
                    logging.info("다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지일 수 있습니다.")
                    break
            except (TimeoutException, NoSuchElementException) as e:
                logging.error(f"다음 페이지 버튼을 찾을 수 없습니다. 오류: {e}")
                break

        except TimeoutException:
            logging.error("페이지 로드 시간이 초과되었습니다. 다음 페이지로 넘어갑니다.")
            page_num += 1
            continue
        except NoSuchWindowException:
            logging.error("브라우저 창이 닫혔습니다. 프로그램을 종료합니다.")
            break
        except Exception as e:
            logging.error(f"예상치 못한 오류가 발생했습니다: {e}")
            break
    
    logging.info(f"총 {len(reviews_data)}개의 리뷰를 수집했습니다.")

# 파일을 시스템에서 자동으로 여는 함수
def open_file(file_path):
    if os.name == 'nt':  # Windows
        os.startfile(file_path)
    elif os.name == 'posix':  # macOS and Linux
        if sys.platform == 'darwin':  # macOS
            subprocess.call(('open', file_path))
        else:  # Linux
            subprocess.call(('xdg-open', file_path))

# 메인 함수
def main():
    excel_file_path = input("입력 엑셀 파일의 경로를 입력하세요: ")
    if not os.path.exists(excel_file_path):
        logging.error("입력한 엑셀 파일이 존재하지 않습니다.")
        return

    inputs = read_excel_input(excel_file_path)
    if not inputs:
        logging.error("엑셀 파일에서 올바른 입력을 읽어올 수 없습니다.")
        return

    for index, (url, file_name, sort_option, max_reviews) in enumerate(inputs):
        logging.info(f"처리 중: {url}")
        driver = initialize_webdriver()
        driver.get(url)

        try:
            check_and_click_refresh_button(driver)
            
            review_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-name="REVIEW"]'))
            )
            review_tab.click()
            random_wait()

            set_review_sort(driver, sort_option)

            reviews_data = []
            extract_review_info(driver, reviews_data, max_reviews, file_name, sort_option)
            
            if reviews_data:
                file_path = save_to_csv_partial(reviews_data, file_name, sort_option, 1)
                logging.info(f"{len(reviews_data)}개의 리뷰를 저장했습니다.")
                upload_to_google_drive(file_path, GOOGLE_DRIVE_FOLDER_ID)
                update_excel_status(excel_file_path, index, "완료")
                open_file(file_path)
            else:
                logging.warning("수집된 리뷰가 없습니다.")
                update_excel_status(excel_file_path, index, "실패")

        except Exception as e:
            logging.error(f"처리 중 오류 발생: {e}")
            update_excel_status(excel_file_path, index, "실패")
        
        finally:
            driver.quit()

    logging.info("모든 작업이 완료되었습니다.")

if __name__ == "__main__":
    main()
    

    # 엑셀화일 경로
    # C:\Users\owner\Documents\python\엑셀업로드양식(스마트스토어).xlsx


입력 엑셀 파일의 경로를 입력하세요: C:\Users\owner\Documents\python\엑셀업로드양식(스마트스토어).xlsx


2025-03-19 17:47:51,941 - INFO - 엑셀 파일에서 읽어온 열 이름: ['상품 페이지 URL', '파일 이름', '정렬 옵션', '최대 리뷰 수']
2025-03-19 17:47:51,954 - INFO - 처리 중: https://smartstore.naver.com/sevy/products/9677920440
2025-03-19 17:47:51,963 - INFO - ====== WebDriver manager ======
2025-03-19 17:47:54,410 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-19 17:47:54,782 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-19 17:47:55,100 - INFO - Driver [C:\Users\owner\.wdm\drivers\chromedriver\win64\134.0.6998.88\chromedriver-win32/chromedriver.exe] found in cache
2025-03-19 17:48:07,803 - INFO - 새로고침 버튼이 감지되지 않았습니다. 계속 진행합니다.
2025-03-19 17:48:11,861 - INFO - 선택한 정렬 옵션: 최신순
2025-03-19 17:48:14,378 - INFO - 현재 페이지: 1
2025-03-19 17:48:14,388 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2025-03-19 17:48:14,449 - INFO - 리뷰 1: ange****** - 4
2025-03-19 17:48:14,490 - INFO - 리뷰 2: ange****** - 4
2025-03-19 17:48:14,538 - INFO - 리뷰 3: ange****** - 4
2025-03-19 17:48:14,578 - INFO - 리뷰 4: myle**** - 5
202

2025-03-19 17:48:39,839 - INFO - 리뷰 127: ko**** - 5
2025-03-19 17:48:39,879 - INFO - 리뷰 128: fn**** - 5
2025-03-19 17:48:39,918 - INFO - 리뷰 129: fn**** - 5
2025-03-19 17:48:39,957 - INFO - 리뷰 130: fn**** - 5
2025-03-19 17:48:39,996 - INFO - 리뷰 131: by**** - 5
2025-03-19 17:48:40,037 - INFO - 리뷰 132: 6893*** - 5
2025-03-19 17:48:40,086 - INFO - 리뷰 133: 6893*** - 5
2025-03-19 17:48:40,128 - INFO - 리뷰 134: 6893*** - 5
2025-03-19 17:48:40,172 - INFO - 리뷰 135: 6893*** - 5
2025-03-19 17:48:40,218 - INFO - 리뷰 136: eun4*** - 5
2025-03-19 17:48:40,272 - INFO - 리뷰 137: ceci*** - 5
2025-03-19 17:48:40,314 - INFO - 리뷰 138: sk**** - 5
2025-03-19 17:48:40,355 - INFO - 리뷰 139: sk**** - 5
2025-03-19 17:48:40,399 - INFO - 리뷰 140: sk**** - 5
2025-03-19 17:48:42,476 - INFO - 현재 페이지: 8
2025-03-19 17:48:42,483 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2025-03-19 17:48:42,522 - INFO - 리뷰 141: sk**** - 5
2025-03-19 17:48:42,563 - INFO - 리뷰 142: wjdd*** - 5
2025-03-19 17:48:42,603 - INFO - 리뷰 143: cooo***** - 5
2025-03-